This preprocessing file cleans up the survey and reference data files and creates reference files in the resources folder for the analysis carried out in the 2020 descriptive notebook.

In [1]:
# import statements

import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import datetime as dt
import math
import json
import csv


# set useful variables for accessing the files

here = os.getcwd()
flora_h = F"{here}/resources/reference-data/"
flora_h_ws = F"{flora_h}atlasws/"
flora_h_55 = F"{flora_h}atlas5x5/"
data_2020 = F"{here}/resources/survey-data/"

In [2]:
# importing the data files

# start by organizing them into dictionaries.

my_data_methods = {"csv":pd.read_csv}

# dict for the 2020 survey data files
d_files = {
    "surveys":"surveys.csv",
    "map_keys":"map-keys.csv",    
}

# dict for the reference files that are not Welten Sutter or Flora Helvetica 5X5 lists

w_lists = {
    "list_2014":"BL_WL_2014_modified.csv",
    "under_sampled":"taxa_sous_echantillonnes.csv",
    "red_list":"redlist2019.csv",
    "cert_list": "Certification_specieslist_2021.csv"
}

# dict for the Welten-Sutter map reference files, downloaded from here: https://www.infoflora.ch/de/daten/artenliste-welten-sutter.html
# all observations included in this report were conducted within one of these geographic boundaries

ws_lists ={
    "151":"AtlasWS_151_Biel.csv",
    "252":"AtlasWS_252_Erlach.csv",
    "300":"AtlasWS_300_Aarberg.csv",
    "301":"AtlasWS_301_Bueren.csv",
    "154":"AtlasWS_154_Grenchen.csv",
    "572":"AtlasWS_572_Beatenberg.csv",
    "573":"AtlasWS_573_Interlaken.csv",
    "226":"AtlasWS_226_Estavayer.csv",
    "251":"AtlasWS_251_BernWest.csv",
    "145":"AtlasWS_145_LesRangiers.csv"
}


# dict for the Flora Helvetica 5x5 map reference files, downloaded from here: https://www.infoflora.ch/de/daten/artenliste-5x5-km.html
# all observations included in this report were conducted within one of these geographic boundaries

# housekeeping: 585220 is separated by "," not ";" like the rest of the data sources
df = pd.read_csv("resources/reference-data/atlas5x5/Atlas5x5_585_220.csv", sep = ",", encoding="utf-16")
df.to_csv('resources/reference-data/atlas5x5/Atlas5x5_585_220_1.csv', sep=';', encoding = "utf-16", index = False)

fx_lists = {
    "585215":"Atlas5x5_585_215.csv", # Ipsach, Bielersee
    "585220":"Atlas5x5_585_220_1.csv", # Biel Stadt, Suze / Bielersee
    "580220":"Atlas5x5_580_220.csv", # Biel Mett, Suze
    "580215":"Atlas5x5_580_215.csv", # Port, Nidau-Bueren Kanal
    "625165":"Atlas5x5_625_165.csv", # Untersee, Thunersee
    "625170":"Atlas5x5_625_170.csv", # Sundlauenen, Thunersee
    "550185":"Atlas5x5_550_185.csv", # Estavayer, Lac de Neuchatel
    "575210":"Atlas5x5_575_210.csv", # Leuecherz, Bielersee
    "600200":"Atlas5x5_600_200.csv", # Bern west, Aare
    "575245":"Atlas5x5_575_245.csv",  # Saint-Ursanne, Aare
    "545180":"Atlas5x5_545_180.csv",
    "575215":"Atlas5x5_575_215.csv"
}

# convenience method to gather up all the files:

def get_the_data(file_exts, a_dir, methods, this_method="csv", myencoding=None):
    wiw = {}
    for k,v in file_exts.items():
        if myencoding == None:
            wiw.update({k:methods[this_method](F"{a_dir}{v}")})            
        else:
            wiw.update({k:methods[this_method](F"{a_dir}{v}",sep = ";", encoding=myencoding)})
    return wiw

# use the get_the_data method to collect these files

data_and_keys = get_the_data(d_files, data_2020, my_data_methods, this_method="csv")
watch_lists = get_the_data(w_lists, flora_h, my_data_methods, this_method="csv")
welt_sut =  get_the_data(ws_lists, flora_h_ws, my_data_methods, this_method="csv", myencoding = "utf-16" )
fivex =  get_the_data(fx_lists, flora_h_55, my_data_methods, this_method="csv", myencoding = "utf-16")

Organization of the reference files

In [3]:
# housekeeping

# housekeeping: make sure that each data set has the column "species", with the value species:
watch_lists["list_2014"]["species"] = watch_lists["list_2014"].Latin
watch_lists["under_sampled"]["species"] = watch_lists["under_sampled"].taxon
watch_lists["red_list"]["species"] = watch_lists["red_list"].scientific_name
watch_lists["cert_list"]["species"] = watch_lists["cert_list"]["Short Name"]


In [4]:
# make a species slug (genus-species) to link data from across the survey and reference files. 
# This is necessary as some species columns have only "Genus species", some include subspecies, and some include the taxonomic reference.

# function to make the species slugs
def to_species_slug(x):
    try: 
        int_data = x.split()
        data = int_data[:2]
        data = "-".join(data)
        data = data.lower()
    except:
        data = "none"
    return data

# create a new column to hold the slug

for element in [fivex, welt_sut, watch_lists]:
    for the_data in element:
        element[the_data]['species_slug'] = 'none'

# make the species slug for all reference files

for element in [fivex, welt_sut, watch_lists]:
    for the_data in element:
        element[the_data]['species_slug'] = element[the_data].species.map(lambda x: to_species_slug(x))

In [5]:
# Add identifying columns to the reference datasets

# add a column to identify the map source for the geographic data:

for element in [fivex, welt_sut]:
    for the_data in element:
        element[the_data]['map'] = the_data
        element[the_data]['spec_map'] = list(zip(element[the_data].species_slug,element[the_data].map))
        
# add a column to identify watch list:

for element in [watch_lists]:
    for the_data in element:
        if the_data == "list_2014":
            element[the_data]['watch_list'] = element[the_data][the_data]
        else:
            element[the_data]['watch_list'] = the_data

# housekeeping: fill in nan values in the watchlist and certification list reference files.

fill_nans = watch_lists["list_2014"].copy()
fill_nans = fill_nans.fillna(0)
watch_lists.update({"list_2014":fill_nans[fill_nans.watch_list != 0]})

fill_nans = watch_lists["cert_list"].copy()
fill_nans = fill_nans.fillna(0)
watch_lists.update({"cert_list":fill_nans[fill_nans.watch_list != 0]})


### Species name and observations: harmonizing taxonomy

The genus-species nomenclature will be used to group observations.

All observations will be classified according to that standard. As a result subspecies will be folded in with the parent species. This is a reflection of the survey method and the expectation of reasonable results, not a prioritization of importance.

In [6]:
def account_for_subspecies(an_array, a_dict):
    for element in an_array:
        try:
            a_dict[element[0]].append(element[1])
        except:
            a_dict[element[0]] = [element[1]]
    return a_dict
a_dict ={}

for element in [fivex, welt_sut, watch_lists]:
    for label in element:
        # use this data frame
        som_data = element[label].copy()
        
        # group by species slug and count the number of unique species values
        c_s_p_s = som_data.groupby('species_slug', as_index=False).species.nunique()        
        
        # just the records with more than one species value
        m_t_one = c_s_p_s[c_s_p_s.species > 1].species_slug
        
        # pair the species_slug to the species name:
        mto = som_data.loc[som_data.species_slug.isin(m_t_one)][['species_slug', 'species']].copy().to_numpy()
        
        # update the dict
        account_for_subspecies(mto, a_dict)

# the species_slugs that account for more than one sub species
sub_species_accounted =  {k:list(set(v)) for k,v in a_dict.items()}

# the species_slugs
gs_parent = list(sub_species_accounted.keys())

# the number of species_slugs
number_of_gs = len(gs_parent)

# the the number of sub species accounted for
number_of_ss = sum([len(v) for k,v in sub_species_accounted.items()])

### Collect welten sutter and 5x5 observations and export to a json dict.

In [7]:
# collect all the observations into one df
fx = pd.concat([v[['species_slug', 'map', 'spec_map']] for k,v in fivex.items()])
wsx = pd.concat([v[['species_slug', 'map', 'spec_map']] for k,v in welt_sut.items()])
f_w_obs = pd.concat([fx, wsx])

In [8]:
# gather up the map names
m_ap_columns = f_w_obs.map.unique()

# create a column for each map, indicate
for col in m_ap_columns:    
    f_w_obs[col] = f_w_obs['map'] == col

obs_map =f_w_obs.groupby(['species_slug']).sum()

obs_map['maps'] = obs_map.index.map(lambda x: f_w_obs[f_w_obs.species_slug == x]['map'].unique())

In [9]:
#exports the dictionary to a .json file
nt = obs_map['maps'].to_dict()

ntx = {k:list(v) for k,v in nt.items()}

with open(F"{here}/output/ws_list.json","w") as afile:
    json.dump(ntx,afile)
    
print(F"\nWhich maps 'trifolium-incarnatum' were found in? indifferent of subspecies?:\n\n{ntx['trifolium-incarnatum']}\n")


Which maps 'trifolium-incarnatum' were found in? indifferent of subspecies?:

['585215', '585220', '580215', '600200', '545180', '151', '252', '300', '154', '226', '251', '145']



In [10]:
# use red_list in w_lists to identify the status of species encountered.
# use the status in the "CH" column for "n"
# use the below dictionary to translate the status

# combine this with watch list / black list data from list_2014 in watch_lists and export to .json dictionary.
# not this

endangerdict = {
    "LC": "Least Concern",
    "NT": "Near Threatened",
    "VU": "Vulnerable",
    "EN": "Endangered",
    "CR": "Critically Endangered"
}

# map this to
# df.this_key.map(lamda x: region_dict[x])

regiondict = {
    "JU": "Jura",
    "MP": "Central Plateau",
    "CH": "National"
}

In [11]:
# format scientific name in the sample dataset

samples = data_and_keys['surveys'].copy()

# dictionary of replacement values that are incorrect

replacedict = {
    'verbanum bonariensis ':'verbena bonariensis',
    'medicago varia':'medicago sativa',
    "oenothera":"oenothera biennis",
    "geranium pratens":"geranium pratense",
    "senecio jacobaea": "jacobaea vulgaris",
    "oenothera biennis ": "oenothera biennis",
    "oenothera biennis agg.": "oenothera biennis",
    "solidalgo canadensis": "solidago canadensis",
    "verbascum lynchitis":"verbascum lychnitis",
    "verbascum negris":"verbascum nigrum",
    "securigea varia": "securigera varia",
    "melilotus officianalis": "melilotus officinalis",
    "knautia maxima": "knautia dipsacifolia",
    "hieracium aurantiacum":"pilosella aurantiaca",
    "sysimbrium officinale":"sisymbrium officinale",
    "geranium robertanium":"geranium robertianum",
    "mycelis muralis": "lactuca muralis",
    "calamintha-nepeta":"clinopodium nepeta",
    "polygonum-persicaria":"persicaria maculosa",
    "sorbus-aria":"aria edulis",
    "taraxacum": "taraxacum officinale"
}

# function to assign the correct value of the key is in the samples dictionary.

def new_func(x,keys):
    try:
        data = keys[x]
    except:
        data = x
    return data

# apply the funtion to a copy of the surveys data set.

samples["species2"] = samples.sci.map(lambda x: new_func(x, replacedict))
samples["species_slug"] = samples.species2.map(lambda x: to_species_slug(x))

# update the surveys dataset.

data_and_keys.update({'surveys':samples})

In [12]:
# format dates in the sample dataset

# function converts DD.MM.YYYY format to YYYY-MM-DD format, ignores if already in YYYY-MM-DD format

def change_string(x):
    try:
        s_data = x.split('.')
        data = s_data[::-1]
        data = "-".join(data)
    except:
        print("no luck")
        data = x
    
    return data

# applies the function to a column in the samples data frame

samples['new_date'] = samples.date.map(lambda x: change_string(x))

# function makes a timestamp out of the YYYY-MM-DD string.

def make_timestamp(x):
    try:        
        data = dt.datetime.strptime(x, "%Y-%m-%d")        
    except:        
        data = 'no luck'
    
    return data

# run the make_timestamp function and store the results in the samples dataframe.

samples['stamp_date'] = samples.new_date.map(lambda x: make_timestamp(x))
samples['date'] = samples.stamp_date

In [13]:
# make place name slugs in the sample dataset

#  change_place function turns "place names 1" into place-names-1
def change_place(x):
    data = x.split(" ")
    data = "-".join(data)
    return data
samples["place1"] = samples.place.map(lambda x: change_place(x))

In [14]:
# clean up redundant and unused column names

# rename the place_slug column

samples.rename(columns= {'place1':'place_slug'}, inplace=True)

# remove the unnecessary columns for this analysis

samples.drop(['species2', 'stamp_date', 'place', 'sci', 'time', 'name'] , inplace=True, axis=1)

samples['loc_date'] = list(zip(samples.place_slug, samples.date))

In [15]:
# export samples to a .csv for later use
samples.to_csv(F"resources/preprocessed/hd_samples_2020.csv", index=False)

In [16]:
samples.head()

,date,species_slug,new_date,place_slug,loc_date
0,2020-09-02,plantago-lanceolata,2020-09-02,alleestrasse-1,"(alleestrasse-1, 2020-09-02 00:00:00)"
1,2020-09-02,centaurea-nigra,2020-09-02,alleestrasse-1,"(alleestrasse-1, 2020-09-02 00:00:00)"
2,2020-09-02,plantago-media,2020-09-02,alleestrasse-1,"(alleestrasse-1, 2020-09-02 00:00:00)"
3,2020-09-02,chenopodium-album,2020-09-02,alleestrasse-1,"(alleestrasse-1, 2020-09-02 00:00:00)"
4,2020-09-02,centaurea-jacea,2020-09-02,alleestrasse-1,"(alleestrasse-1, 2020-09-02 00:00:00)"
